In [1]:
import pandas as pd
import requests
import re

# Extract CSVs into DataFrames

In [2]:
yelp_file = "../Resources/yelp_data.csv"
yelp_df = pd.read_csv(yelp_file)
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",NaN
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",NaN
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",NaN
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


In [3]:
# All columns
yelp_df.columns

Index(['restaurant_name', 'yelp_url', 'restaurant_rating', 'price_cuisine',
       'address1', 'address2', 'restaurant_website'],
      dtype='object')

In [4]:
# restaurant_name check
yelp_df["restaurant_name"].value_counts()


Sandwiches Unlimited Lunch Box    2
Chef Fredy’s Table                1
Sabretti’s Hot Dogs               1
Ruth’s Chris Steak House          1
Sammy’s Ye Old Cider Mill         1
                                 ..
Polo’s Bar & Grill                1
Eagle Mini Diner                  1
Market Tavern                     1
La Forchetta                      1
Madame Thai Cuisine               1
Name: restaurant_name, Length: 237, dtype: int64

# Dropping duplicate restaurant_name and also address1 value

In [5]:
yelp_df= yelp_df.drop_duplicates(subset=['restaurant_name', 'address1'])
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",NaN
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",NaN
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",NaN
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


In [6]:
# address2 check
yelp_df["address2"].value_counts()

Morristown, NJ 07960              32
Randolph, NJ 07869                28
Dover, NJ 07801                   25
Denville, NJ 07834                22
Rockaway, NJ 07866                19
Morris Plains, NJ 07950           16
Wharton, NJ 07885                 13
Ledgewood, NJ 07852               11
Succasunna, NJ 07876               9
Chester, NJ 07930                  8
Mendham, NJ 07945                  7
Lake Hopatcong, NJ 07849           6
Flanders, NJ 07836                 5
Stanhope, NJ 07874                 4
Netcong, NJ 07857                  4
Mine Hill, NJ 07803                4
Hopatcong, NJ 07843                3
Mount Arlington, NJ 07856          2
Cedar Knolls, NJ 07927             2
Roxbury Township, NJ 07852         2
Mount Olive Township, NJ 07836     2
Kenvil, NJ 07847                   2
Ironia, NJ 07845                   1
Parsippany, NJ 07054               1
Mountain Lakes, NJ 07046           1
Rockaway Township, NJ 07866        1
Rockaway, NJ 07801                 1
M

In [7]:
# average rating
average_rating = yelp_df["restaurant_rating"].sum()/yelp_df["restaurant_name"].count()
print("average_rating")
print(average_rating)


average_rating
3.9936974789915967


In [8]:
# There are different length for restaurant_name and address1 values.So,we need to fill the empty values.
yelp_df=yelp_df.fillna(0)
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",0
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",0
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",0
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


In [9]:
# restaurant_rating check
yelp_df["restaurant_rating"].count()

238

# Drop a row if it contains a certain value (in this case, “com” or "net")

In [10]:
# restaurant_url check
yelp_df["restaurant_website"].value_counts()

0                                  52
Find a Table                        3
sandwichesunlimitedlunchbox.co…     2
anthonyandsonsbakery.com            2
malaymorristown.com                 1
                                   ..
thatchermcghees.com/denville        1
lovinghut.us                        1
veggieheavennj.com                  1
mammacaterinapizza.com              1
polosbarandgrill.com                1
Name: restaurant_website, Length: 183, dtype: int64

In [11]:
#new_yelp_df1=new_yelp_df[new_yelp_df.restaurant_website != 'Find a Table']
#new_yelp_df1


In [12]:
#new_yelp_df2=new_yelp_df1[new_yelp_df1.restaurant_website != 0]
#new_yelp_df2

In [13]:
# restaurant_website check
#new_yelp_df2["restaurant_website"].value_counts()

In [14]:
#new_yelp_df3=new_yelp_df2.reset_index()
#new_yelp_df3

# Split the price_cuisine column in to price_level and cuisine column

In [15]:
# price_cuisine check
yelp_df["price_cuisine"].value_counts()

$$Italian                                  12
$$Thai                                      6
$Chinese                                    5
$$Mexican                                   5
$$Indian                                    5
                                           ..
$$$Steakhouses, Seafood, Wine Bars          1
$$Japanese, Korean, Sushi Bars              1
$$$Italian, Bars, Venues & Event Spaces     1
$$$American (New), French, Salad            1
$$$Italian, Wine Bars, Cafes                1
Name: price_cuisine, Length: 183, dtype: int64

In [16]:
price_level = []
cuisine = []
for index, row in yelp_df.iterrows():
    price_cuisine = re.split(r'(\$+)', row['price_cuisine'])
    if len(price_cuisine) == 1:
        price_level.append("")
        cuisine.append(price_cuisine[0])
    else:
        price_level.append(price_cuisine[1])
        cuisine.append(price_cuisine[2])
yelp_df['price_level'] = price_level
yelp_df['cuisine'] = cuisine
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website,price_level,cuisine
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table,$$$,"Italian, Bars, Venues & Event Spaces"
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com,$$,"American (New), Bars, Pizza"
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com,$$,"Wine Bars, French, Italian"
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates"
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com,$$,Italian
...,...,...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",0,$,"Food Trucks, American (Traditional), Hot Dogs"
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",0,$,Pizza
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",0,$,"Juice Bars & Smoothies, Soup, Sandwiches"
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…,,"Juice Bars & Smoothies, Acai Bowls, Wraps"


# Split the Split address2 to city, state and zip code

In [17]:
yelp_df[['city','state+zipcode']]=yelp_df.address2.str.split(",",expand=True,)
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website,price_level,cuisine,city,state+zipcode
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ 07869
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ 07869
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ 07885
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ 07801
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com,$$,Italian,Randolph,NJ 07869
...,...,...,...,...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",0,$,"Food Trucks, American (Traditional), Hot Dogs",Dover,NJ 07801
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",0,$,Pizza,Succasunna,NJ 07876
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",0,$,"Juice Bars & Smoothies, Soup, Sandwiches",Dover,NJ 07801
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…,,"Juice Bars & Smoothies, Acai Bowls, Wraps",Morristown,NJ 07960


In [18]:
yelp_df["state+zipcode"]=yelp_df["state+zipcode"].str.lstrip()

In [19]:
yelp_df[['state', 'zip code']] = yelp_df["state+zipcode"].str.split(" ",expand=True)
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website,price_level,cuisine,city,state+zipcode,state,zip code
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ 07869,NJ,07869
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ 07869,NJ,07869
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ 07885,NJ,07885
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ 07801,NJ,07801
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com,$$,Italian,Randolph,NJ 07869,NJ,07869
...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",0,$,"Food Trucks, American (Traditional), Hot Dogs",Dover,NJ 07801,NJ,07801
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",0,$,Pizza,Succasunna,NJ 07876,NJ,07876
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",0,$,"Juice Bars & Smoothies, Soup, Sandwiches",Dover,NJ 07801,NJ,07801
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…,,"Juice Bars & Smoothies, Acai Bowls, Wraps",Morristown,NJ 07960,NJ,07960


# Clean DataFrame

# all the Zip Codes of Morris County.

In [20]:
tables=pd.read_html('http://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=nj&StName=&StFIPS=&FIPS=34027')
morris_county_df = tables[2]
morris_county_df

,Zip Code,City,County
0,7005,Boonton,Morris County
1,7034,Lake Hiawatha,Morris County
2,7035,Lincoln Park,Morris County
3,7045,Montville,Morris County
4,7046,Mountain Lakes,Morris County
5,7054,Parsippany,Morris County
6,7058,Pine Brook,Morris County
7,7082,Towaco,Morris County
8,7405,Butler,Morris County
9,7405,Kinnelon,Morris County


In [21]:
morris_county_df['Zip Code'] = morris_county_df['Zip Code'].astype(str)
morris_county_zipcodes = morris_county_df['Zip Code'].str.zfill(5)
print(morris_county_zipcodes)
#yelp_df = yelp_df[~yelp_df['zip code'].isin(morris_county_zipcodes)]
yelp_df
yelp_df['zip code'].isin(morris_county_zipcodes)

0     07005
1     07034
2     07035
3     07045
4     07046
5     07054
6     07058
7     07082
8     07405
9     07405
10    07440
11    07444
12    07457
13    07801
14    07802
15    07803
16    07803
17    07806
18    07806
19    07828
20    07834
21    07836
22    07842
23    07845
24    07847
25    07849
26    07850
27    07852
28    07853
29    07856
30    07857
31    07866
32    07869
33    07869
34    07870
35    07876
36    07878
37    07885
38    07926
39    07927
40    07928
41    07930
42    07932
43    07933
44    07935
45    07936
46    07940
47    07945
48    07946
49    07950
50    07950
51    07960
Name: Zip Code, dtype: object


0      True
1      True
2      True
3      True
4      True
       ... 
233    True
234    True
235    True
236    True
237    True
Name: zip code, Length: 238, dtype: bool

In [22]:
yelp_df = yelp_df.drop(['price_cuisine', 'address2', 'state+zipcode'], axis=1)
yelp_df = yelp_df.rename(columns={'restaurant_rating':'yelp_rating',
                                  'price_level':'yelp_price_level',
                                  'cuisine':'yelp_cuisine'
                                 })
yelp_df['restaurant_name'] = yelp_df['restaurant_name'].str.strip()
yelp_df['address1'] = yelp_df['address1'].str.strip()

yelp_df

,restaurant_name,yelp_url,yelp_rating,address1,restaurant_website,yelp_price_level,yelp_cuisine,city,state,zip code
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,1181 Sussex Tpke,Find a Table,$$$,"Italian, Bars, Venues & Event Spaces",Randolph,NJ,07869
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,500 NJ-10,sbknj.com,$$,"American (New), Bars, Pizza",Randolph,NJ,07869
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,322 S Main St,4seasonswharton.com,$$,"Wine Bars, French, Italian",Wharton,NJ,07885
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,64 E Mcfarlan St,quietmanpub.com,$$,"Irish Pub, Seafood, Tapas/Small Plates",Dover,NJ,07801
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,1171 Sussex Tpke,veronarestaurant.com,$$,Italian,Randolph,NJ,07869
...,...,...,...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,Dover Train Station,0,$,"Food Trucks, American (Traditional), Hot Dogs",Dover,NJ,07801
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,275 State Rt 10 E,0,$,Pizza,Succasunna,NJ,07876
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,6B S Warren St,0,$,"Juice Bars & Smoothies, Soup, Sandwiches",Dover,NJ,07801
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,68 South St,cleanjuice.com/locations/morri…,,"Juice Bars & Smoothies, Acai Bowls, Wraps",Morristown,NJ,07960


# Store CSV data into a DataFrame

In [23]:
csv_file = "../Resources/google_data.csv"
google_data_df = pd.read_csv(csv_file)
google_data_df

,Unnamed: 0,restaurant_name,place_id,price_level,restaurant_rating,google_maps_url,address,restaurant_website
0,0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,https://maps.google.com/?cid=15487458085899557598,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,https://www.popeyes.com/store-locator/store/re...
1,1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,https://maps.google.com/?cid=8506076560359997359,"343 Mt Hope Ave, Rockaway, NJ 07866, USA",https://www.buffalowildwings.com/en/locations/...
2,2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,https://maps.google.com/?cid=11876493694413647283,"301 Mt Hope Ave Suite 1048, Rockaway, NJ 07866...",https://www.auntieannes.com/nj/rockaway/119-1?...
3,3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,https://maps.google.com/?cid=1798990059960816432,"510 Tabor Rd, Morris Plains, NJ 07950, USA",http://www.taborroadtavern.com/
4,4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,https://maps.google.com/?cid=9758189877393522658,"559 E Main St Store # 2, Denville, NJ 07834, USA",https://order.subway.com/en-us/restaurant/2197...
...,...,...,...,...,...,...,...,...
355,355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,https://maps.google.com/?cid=1646938978706323838,"15 NJ-10, Succasunna, NJ 07876, USA",http://anthonyandsonsbakerynj.com/
356,356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,https://maps.google.com/?cid=17474442911624320363,"312 Howard Blvd, Mt Arlington, NJ 07856, USA",https://pizzaexpressmtarlington.com/
357,357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,https://maps.google.com/?cid=11162395169597474812,"274 US-46, Mine Hill Township, NJ 07803, USA",https://find.shell.com/us/fuel/12395193-274-rt...
358,358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1.0,4.2,https://maps.google.com/?cid=11489687353021894176,"274 Rte 46, Mine Hill Township, NJ 07803, USA",https://smart.link/abu8tr12sc1y6?utm_source=go...


In [24]:
# All columns
google_data_df.columns

Index(['Unnamed: 0', 'restaurant_name', 'place_id', 'price_level',
       'restaurant_rating', 'google_maps_url', 'address',
       'restaurant_website'],
      dtype='object')

In [25]:
# restaurant name check
google_data_df["restaurant_name"].value_counts()

Subway                            12
Dunkin'                           11
Starbucks                         10
Panera Bread                       4
Wendy's                            4
                                  ..
Yummy Taste Chinese Restaurant     1
Thai Nam Phet                      1
Bryans Luncheonette                1
RJ's Pizzeria                      1
Wasabi Japanese Restaurant         1
Name: restaurant_name, Length: 245, dtype: int64

# Dropping duplicate restaurant address and restaurant_name value


In [28]:
google_data_df= google_data_df.drop_duplicates(subset=['restaurant_name', 'address'])
google_data_df

,Unnamed: 0,restaurant_name,place_id,price_level,restaurant_rating,google_maps_url,address,restaurant_website
0,0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,https://maps.google.com/?cid=15487458085899557598,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,https://www.popeyes.com/store-locator/store/re...
1,1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,https://maps.google.com/?cid=8506076560359997359,"343 Mt Hope Ave, Rockaway, NJ 07866, USA",https://www.buffalowildwings.com/en/locations/...
2,2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,https://maps.google.com/?cid=11876493694413647283,"301 Mt Hope Ave Suite 1048, Rockaway, NJ 07866...",https://www.auntieannes.com/nj/rockaway/119-1?...
3,3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,https://maps.google.com/?cid=1798990059960816432,"510 Tabor Rd, Morris Plains, NJ 07950, USA",http://www.taborroadtavern.com/
4,4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,https://maps.google.com/?cid=9758189877393522658,"559 E Main St Store # 2, Denville, NJ 07834, USA",https://order.subway.com/en-us/restaurant/2197...
...,...,...,...,...,...,...,...,...
355,355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,https://maps.google.com/?cid=1646938978706323838,"15 NJ-10, Succasunna, NJ 07876, USA",http://anthonyandsonsbakerynj.com/
356,356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,https://maps.google.com/?cid=17474442911624320363,"312 Howard Blvd, Mt Arlington, NJ 07856, USA",https://pizzaexpressmtarlington.com/
357,357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,https://maps.google.com/?cid=11162395169597474812,"274 US-46, Mine Hill Township, NJ 07803, USA",https://find.shell.com/us/fuel/12395193-274-rt...
358,358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1.0,4.2,https://maps.google.com/?cid=11489687353021894176,"274 Rte 46, Mine Hill Township, NJ 07803, USA",https://smart.link/abu8tr12sc1y6?utm_source=go...


# Clean DataFrame

In [29]:
# price_level check
google_data_df["price_level"].value_counts()

2.0    134
1.0     78
3.0     21
4.0      2
Name: price_level, dtype: int64

In [30]:
# restaurant_rating check
google_data_df["restaurant_rating"].value_counts()

4.2    47
4.3    44
4.5    39
4.4    39
4.1    26
4.6    20
4.0    14
4.7    13
3.9     6
3.4     5
3.8     5
3.7     5
3.6     4
4.8     3
5.0     3
3.3     2
3.5     2
2.5     2
2.9     1
3.2     1
3.0     1
2.0     1
3.1     1
Name: restaurant_rating, dtype: int64

In [31]:
google_data_df=google_data_df.fillna(0)
google_data_df

,Unnamed: 0,restaurant_name,place_id,price_level,restaurant_rating,google_maps_url,address,restaurant_website
0,0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,https://maps.google.com/?cid=15487458085899557598,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,https://www.popeyes.com/store-locator/store/re...
1,1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,https://maps.google.com/?cid=8506076560359997359,"343 Mt Hope Ave, Rockaway, NJ 07866, USA",https://www.buffalowildwings.com/en/locations/...
2,2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,https://maps.google.com/?cid=11876493694413647283,"301 Mt Hope Ave Suite 1048, Rockaway, NJ 07866...",https://www.auntieannes.com/nj/rockaway/119-1?...
3,3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,https://maps.google.com/?cid=1798990059960816432,"510 Tabor Rd, Morris Plains, NJ 07950, USA",http://www.taborroadtavern.com/
4,4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,https://maps.google.com/?cid=9758189877393522658,"559 E Main St Store # 2, Denville, NJ 07834, USA",https://order.subway.com/en-us/restaurant/2197...
...,...,...,...,...,...,...,...,...
355,355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,https://maps.google.com/?cid=1646938978706323838,"15 NJ-10, Succasunna, NJ 07876, USA",http://anthonyandsonsbakerynj.com/
356,356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,https://maps.google.com/?cid=17474442911624320363,"312 Howard Blvd, Mt Arlington, NJ 07856, USA",https://pizzaexpressmtarlington.com/
357,357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,https://maps.google.com/?cid=11162395169597474812,"274 US-46, Mine Hill Township, NJ 07803, USA",https://find.shell.com/us/fuel/12395193-274-rt...
358,358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1.0,4.2,https://maps.google.com/?cid=11489687353021894176,"274 Rte 46, Mine Hill Township, NJ 07803, USA",https://smart.link/abu8tr12sc1y6?utm_source=go...


# Split the Split address to address, city, state, zip code and country

In [32]:
google_data_df[['address1','city','state_zip','country']] = google_data_df['address'].str.rsplit(',', 3, expand=True)
google_data_df['state_zip'] = google_data_df['state_zip'].str.lstrip()
google_data_df[['state','zip code']] = google_data_df['state_zip'].str.split(' ', 1, expand=True)
google_data_df

,Unnamed: 0,restaurant_name,place_id,price_level,restaurant_rating,google_maps_url,address,restaurant_website,address1,city,state_zip,country,state,zip code
0,0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,https://maps.google.com/?cid=15487458085899557598,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,https://www.popeyes.com/store-locator/store/re...,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,Rockaway,NJ 07866,USA,NJ,07866
1,1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,https://maps.google.com/?cid=8506076560359997359,"343 Mt Hope Ave, Rockaway, NJ 07866, USA",https://www.buffalowildwings.com/en/locations/...,343 Mt Hope Ave,Rockaway,NJ 07866,USA,NJ,07866
2,2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,https://maps.google.com/?cid=11876493694413647283,"301 Mt Hope Ave Suite 1048, Rockaway, NJ 07866...",https://www.auntieannes.com/nj/rockaway/119-1?...,301 Mt Hope Ave Suite 1048,Rockaway,NJ 07866,USA,NJ,07866
3,3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,https://maps.google.com/?cid=1798990059960816432,"510 Tabor Rd, Morris Plains, NJ 07950, USA",http://www.taborroadtavern.com/,510 Tabor Rd,Morris Plains,NJ 07950,USA,NJ,07950
4,4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,https://maps.google.com/?cid=9758189877393522658,"559 E Main St Store # 2, Denville, NJ 07834, USA",https://order.subway.com/en-us/restaurant/2197...,559 E Main St Store # 2,Denville,NJ 07834,USA,NJ,07834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,https://maps.google.com/?cid=1646938978706323838,"15 NJ-10, Succasunna, NJ 07876, USA",http://anthonyandsonsbakerynj.com/,15 NJ-10,Succasunna,NJ 07876,USA,NJ,07876
356,356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,https://maps.google.com/?cid=17474442911624320363,"312 Howard Blvd, Mt Arlington, NJ 07856, USA",https://pizzaexpressmtarlington.com/,312 Howard Blvd,Mt Arlington,NJ 07856,USA,NJ,07856
357,357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,https://maps.google.com/?cid=11162395169597474812,"274 US-46, Mine Hill Township, NJ 07803, USA",https://find.shell.com/us/fuel/12395193-274-rt...,274 US-46,Mine Hill Township,NJ 07803,USA,NJ,07803
358,358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1.0,4.2,https://maps.google.com/?cid=11489687353021894176,"274 Rte 46, Mine Hill Township, NJ 07803, USA",https://smart.link/abu8tr12sc1y6?utm_source=go...,274 Rte 46,Mine Hill Township,NJ 07803,USA,NJ,07803


In [33]:
# restaurant_website check
google_data_df["restaurant_website"].value_counts()

0                                                                                                                      16
https://www.dominos.com/en/?utm_source=google&utm_medium=loclist&utm_campaign=localmaps                                 4
https://www.outback.com/                                                                                                2
http://www.rootssteakhouse.com/                                                                                         2
http://anthonyandsonsbakerynj.com/                                                                                      2
                                                                                                                       ..
http://trattoriatoscanawhippany.com/                                                                                    1
http://www.flandersritas.com/                                                                                           1
https://smart.link/abu8t

# Please find a way to delete them above